# Cleaning
* Missing Values
    * Detect them by counting them
    * Null them**
    * Interpolate
    * Drop them**
* Inconsistant data (logical errors)
* Outliers
    * Detec using IQR (inter-quartile range), Z-score (normal distribution based approach)
    * Drop them
    * fix them
    * Trancate them (winsorising)
    * Leave them as they are
* Text error
    * Special Characters éèêëēėę
    * Numbers that are considered as text
    * Empty cgaracters
   
    * Different notations Ave. des Champs Elysées and Avenue des CHamps Elysées
    * Mixed case (upper and lower), punctuation marks (!,'".:;etc)
    * Special solutions - (NLP Natural Language Processing)
* Duplicate
* Low variance

In [33]:
import os
import pymysql
from sqlalchemy import create_engine
import pandas as pd

In [34]:
username='root'
host='127.0.0.1'
password='brooklyn452*'
database_name='finance'

In [35]:
engine=create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database_name}')

In [36]:
df=pd.read_csv('/Users/jidekickpush/Downloads/vehicles/vehicles.csv')

In [43]:
null_df=df.isna().sum()

In [44]:
null_df=round(null_df[null_df>0]/df.shape[0]*100,2)

In [45]:
null_df

Series([], dtype: float64)

In [46]:
df[null_df.index].dtypes

Series([], dtype: object)

In [41]:
df[['cylinders','displ']]=df[['cylinders','displ']].fillna(0)

KeyError: "None of [Index(['cylinders', 'displ'], dtype='object')] are in the [columns]"

In [ ]:
df.drive=df.drive.fillna(df.drive.mode()[0])

In [ ]:
def fillmode(x):
    return x.fillna(x.mode()[0])

In [ ]:
df[['eng_drcr','trany','trans_dscr']].apply(fillmode)

In [ ]:
df.isna().sum().sum()

# Inconsistant data

In [ ]:
test=df[(df.cylenders==0)&(df.displ!=0)]
test[['year','make','model'','trany,'drive','fueltype','cylinders','displ']]

In [ ]:
df.loc[test.index,'cylenders']=2

# Outliers

In [23]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,35952.0,2000.716400,10.085290,1984.00,1991.000000,2001.000000,2010.000000,2017.000000
Engine Displacement,35952.0,3.338493,1.359395,0.60,2.200000,3.000000,4.300000,8.400000
Cylinders,35952.0,5.765076,1.755268,2.00,4.000000,6.000000,6.000000,16.000000
Fuel Barrels/Year,35952.0,17.609056,4.467283,0.06,14.699423,17.347895,20.600625,47.087143
City MPG,35952.0,17.646139,4.769349,6.00,15.000000,17.000000,20.000000,58.000000
Highway MPG,35952.0,23.880646,5.890876,9.00,20.000000,24.000000,27.000000,61.000000
Combined MPG,35952.0,19.929322,5.112409,7.00,16.000000,19.000000,23.000000,56.000000
CO2 Emission Grams/Mile,35952.0,475.316339,119.060773,37.00,395.000000,467.736842,555.437500,1269.571429
Fuel Cost/Year,35952.0,1892.598465,506.958627,600.00,1500.000000,1850.000000,2200.000000,5800.000000


In [26]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,35952.0,2000.716400,10.085290,1984.00,1991.000000,2001.000000,2010.000000,2017.000000
Engine Displacement,35952.0,3.338493,1.359395,0.60,2.200000,3.000000,4.300000,8.400000
Cylinders,35952.0,5.765076,1.755268,2.00,4.000000,6.000000,6.000000,16.000000
Fuel Barrels/Year,35952.0,17.609056,4.467283,0.06,14.699423,17.347895,20.600625,47.087143
City MPG,35952.0,17.646139,4.769349,6.00,15.000000,17.000000,20.000000,58.000000
Highway MPG,35952.0,23.880646,5.890876,9.00,20.000000,24.000000,27.000000,61.000000
Combined MPG,35952.0,19.929322,5.112409,7.00,16.000000,19.000000,23.000000,56.000000
CO2 Emission Grams/Mile,35952.0,475.316339,119.060773,37.00,395.000000,467.736842,555.437500,1269.571429
Fuel Cost/Year,35952.0,1892.598465,506.958627,600.00,1500.000000,1850.000000,2200.000000,5800.000000


In [22]:
df.phevComb.plot.box()

'1.0.3'

In [ ]:
stats=df.describe().transpose()

In [ ]:
stats['IQR']=stats['75%']-stats['25%']

In [ ]:
stats

In [ ]:
outliers=pd.DataFrame(columns=df.columns)

In [ ]:
outliers

In [ ]:
for col in stats.index:
    iqr=stats.IQR[col]
    cutoff=iqr*1.6
    lower=stats.at[col,'25%']-cutoff
    upper=stats.at[col,'75%']+cutoff
    result=df[(df.col<lower)|(df.col>upper)].copy()
    results[outliers]=col
    outliers=outliers.append(results)

In [ ]:
outliers

# test error

In [ ]:
df.trany

In [ ]:
df.trany.nunique()

In [ ]:
df.trany.str.replace('-','').str.replace('Automatic','Auto').str.replace('Auto\(',')Auto'.str.replace('\))

In [ ]:
df.trany.str.replace('Manual\(','Manual').str.replace('\)','').str.replace('\(','').unique()

# Duplicates

In [ ]:
print(outliers.shape)
outliers.drop_duplicates()
print(outliers.shape)

In [ ]:
print(outliers.shape)
outliers.drop_duplicates()
print(outliers.iloc[:,:-1].drop_duplicates().shape)

In [ ]:
outliers.iloc[:,:-1].duplicated()